In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
for expression in os.listdir('train/'):
    print(str(len(os.listdir('train/' + expression))) + " " + expression + " images")

4965 neutral images
4097 fear images
4830 sad images
3171 surprise images
3995 angry images
436 disgust images
7214 happy images


In [3]:
img_size = 48
batch_size = 128
epochs = 10

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory('train/',
                                                   target_size = (img_size, img_size),
                                                   batch_size = batch_size,
                                                   color_mode = 'grayscale',
                                                   class_mode = 'categorical',
                                                   shuffle = True)

Found 28708 images belonging to 7 classes.


In [5]:
val_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip=True)
val_generator = val_datagen.flow_from_directory('test/',
                                               target_size = (img_size, img_size),
                                               batch_size = batch_size,
                                               color_mode = 'grayscale',
                                               class_mode = 'categorical',
                                               shuffle = False)

Found 7178 images belonging to 7 classes.


In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization,Conv2D,Dense,Flatten,Dropout,MaxPooling2D,Activation 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

In [7]:
model = Sequential()

model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(7, activation='softmax'))

In [8]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = val_generator.n // val_generator.batch_size

In [10]:
checkpoint = ModelCheckpoint("model_weights.h5",
                            monitor = 'val_accuracy',
                            save_weights_only = True,
                            mode = 'max',
                            verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             patience = 3,
                             factor = 0.1,
                             min_lr = 0.000001)
callbacks = [checkpoint, reduce_lr]

In [11]:
history = model.fit(x = train_generator,
                   validation_data = val_generator,
                   epochs = epochs,
                   steps_per_epoch = steps_per_epoch,
                   validation_steps = validation_steps,
                   callbacks = callbacks)

Epoch 1/10
224/224 [==============================] - ETA: 0s - loss: 1.8801 - accuracy: 0.2723
Epoch 00001: saving model to model_weights.h5
224/224 [==============================] - 713s 3s/step - loss: 1.8801 - accuracy: 0.2723 - val_loss: 2.1877 - val_accuracy: 0.2475
Epoch 2/10
224/224 [==============================] - ETA: 0s - loss: 1.5334 - accuracy: 0.4076
Epoch 00002: saving model to model_weights.h5
224/224 [==============================] - 685s 3s/step - loss: 1.5334 - accuracy: 0.4076 - val_loss: 1.8846 - val_accuracy: 0.2112
Epoch 3/10
224/224 [==============================] - ETA: 0s - loss: 1.3447 - accuracy: 0.4845
Epoch 00003: saving model to model_weights.h5
224/224 [==============================] - 686s 3s/step - loss: 1.3447 - accuracy: 0.4845 - val_loss: 1.4966 - val_accuracy: 0.4488
Epoch 4/10
224/224 [==============================] - ETA: 0s - loss: 1.2540 - accuracy: 0.5215
Epoch 00004: saving model to model_weights.h5
224/224 [===========================

In [12]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [14]:
emotion_dict = {
    0 : 'Angry', 
    1 : 'Disgusted',
    2 : 'Fearful', 
    3 : 'Happy', 
    4 : 'Neutral', 
    5 : 'Sad', 
    6 : 'Surprised'
}

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    if not ret:
        break
    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print('Exit from Loop')
cap.release()
cv2.destroyAllWindows()